In [2]:
from utils.DevConf import DevConf
DEV_CONF = DevConf(device='cuda')

# Define Model

In [1]:
from transformers import AutoTokenizer, DistilBertModel
bertTokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-multilingual-cased")
bertModel = DistilBertModel.from_pretrained("distilbert/distilbert-base-multilingual-cased")
bertModel.eval()

d:\code\python\bert\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): L

In [3]:
from utils.AttnBlocksConf import AttnBlocksConf

In [4]:
from model.BertDecoder.SentiClassifier import SentiClassifier
from model.CombinationModel import CombinationModel
from torch import nn

In [6]:
from utils.const import BlockType

In [7]:
mapper = SentiClassifier(AttnBlocksConf(768, 12, 6), BlockType.PARALLEL)
cModel = CombinationModel(
    distilBert=bertModel,
    decoder=mapper,
    outputProject=nn.Linear(768, 2),
    devConf=DEV_CONF)

# Load Data

In [8]:
from torchtext.datasets import IMDB
from torchtext.data.functional import to_map_style_dataset

In [9]:
trainData = iter(IMDB(root="./data", split='train'))

In [10]:
from torch.utils.data import DataLoader

In [11]:
import torch
from torch import Tensor

In [12]:
def collate_fn(batch)->tuple[Tensor, Tensor, Tensor]:
    target =[]
    datas = []

    for i, (label, data) in enumerate(batch):
        datas.append(data)
        target.append(label-1)

    encoding = bertTokenizer(datas, return_tensors='pt', padding=True, truncation=True, max_length=512).to(DEV_CONF.device)
    ids = encoding['input_ids']
    mask = encoding['attention_mask']
    
    return ids, mask, torch.tensor(target, dtype=torch.int16)

In [13]:
trainLoader = DataLoader(to_map_style_dataset(trainData), collate_fn=collate_fn, batch_size=16, shuffle=True)

In [14]:
next(iter(trainLoader))

(tensor([[  101, 10117, 12648,  ...,     0,     0,     0],
         [  101,   138,   171,  ...,     0,     0,     0],
         [  101, 86804, 14985,  ...,     0,     0,     0],
         ...,
         [  101, 10167, 10105,  ..., 15551, 34420,   102],
         [  101, 10747, 10124,  ...,     0,     0,     0],
         [  101,   146, 24944,  ..., 16700,   131,   102]], device='cuda:0'),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1]], device='cuda:0'),
 tensor([1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1], dtype=torch.int16))

# Train

In [15]:
lr = 1e-5
epochs = 1

In [16]:
loss_fn = nn.CrossEntropyLoss()

In [17]:
optimizer = torch.optim.AdamW(cModel.parameters(), lr=lr)

In [18]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (x, y, z) in enumerate(dataloader):
        # x, y = x.to(devConf.device), y.to(devConf.device)
        z = z.to(DEV_CONF.device).to(torch.long)
        if x.shape[1] <= 512:
            # Compute prediction error
            pred = model(input_ids=x, attention_mask=y)
            loss = loss_fn(pred, z)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(x)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [19]:
cModel.train()
for i in range(epochs):
    print(f"Epoch {i + 1}\n-------------------------------")
    train(trainLoader, cModel, loss_fn, optimizer)
print("Done!")

Epoch 1
-------------------------------


d:\code\python\bert\.venv\Lib\site-packages\torch\nn\functional.py:5476: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


loss: 0.684940  [    0/25000]
loss: 0.660107  [ 1600/25000]
loss: 0.617618  [ 3200/25000]
loss: 0.753508  [ 4800/25000]
loss: 0.626296  [ 6400/25000]
loss: 0.520253  [ 8000/25000]
loss: 0.513562  [ 9600/25000]
loss: 0.472687  [11200/25000]
loss: 0.476501  [12800/25000]
loss: 0.407248  [14400/25000]
loss: 0.721785  [16000/25000]
loss: 0.491819  [17600/25000]
loss: 0.666025  [19200/25000]
loss: 0.516989  [20800/25000]
loss: 0.319325  [22400/25000]
loss: 0.523435  [24000/25000]
Done!


# Evaluate

In [20]:
cModel.eval()

CombinationModel(
  (distilBert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1):

In [21]:
# neg
testInput = bertTokenizer("I think I will make a movie next weekend. Oh wait, I'm working..oh I'm sure I can fit it in. It looks like whoever made this film fit it in. I hope the makers of this crap have day jobs because this film sucked!!! It looks like someones home movie and I don't think more than $100 was spent making it!!! Total crap!!! Who let's this stuff be released?!?!?!",
              return_tensors='pt', padding=True, truncation=True).to(DEV_CONF.device)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [22]:
cModel(**testInput)

tensor([[9.9906e-01, 9.3980e-04]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [25]:
# pos
testInput = bertTokenizer("Before Dogma 95: when Lars used movies as art, not just a story. A beautiful painting about love and death. This is one of my favorite movies of all time. The color... The music... Just perfect."
                          , return_tensors='pt', padding=True, truncation=True).to(DEV_CONF.device)

In [26]:
cModel(**testInput)

tensor([[7.2492e-07, 1.0000e+00]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [27]:
# neg
testInput = bertTokenizer("Ned aKelly is such an important story to Australians but this movie is awful. It's an Australian story yet it seems like it was set in America. Also Ned was an Australian yet he has an Irish accent...it is the worst film I have seen in a long time",
                          return_tensors='pt', padding=True, truncation=True).to(DEV_CONF.device)

In [29]:
cModel(**testInput)

tensor([[0.9946, 0.0054]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

# Save Model

In [30]:
import torch

In [31]:
# torch.save(cModel.state_dict(), "./weights/cModel_state_dict.pth")

In [32]:
torch.save(mapper.state_dict(), "./weights/mapper_state_dict.pth")

In [33]:
# torch.save(bertModel.state_dict(), "./weights/bertModel_state_dict.pth")

In [34]:
torch.save(cModel.outProj.state_dict(), "./weights/outProj_state_dict.pth")

# Load Model

In [35]:
from transformers import DistilBertModel, AutoTokenizer

In [36]:
bertTokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-multilingual-cased")
bertModel = DistilBertModel.from_pretrained("distilbert/distilbert-base-multilingual-cased")
# bertModel.load_state_dict(torch.load("./weights/bertModel_state_dict.pth"))
bertModel.eval()

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): L

In [37]:
from utils.const import BlockType
from utils.DevConf import DevConf
from utils.AttnBlocksConf import AttnBlocksConf
DEV_CONF = DevConf('cpu')

In [38]:
import torch
from torch import nn

In [39]:
testOutput = nn.Linear(768, 2, device=DEV_CONF.device)
testOutput.load_state_dict(torch.load("./weights/outProj_state_dict.pth"))

<All keys matched successfully>

In [40]:
from model.BertDecoder.SentiClassifier import SentiClassifier

In [41]:
testMapper = SentiClassifier(AttnBlocksConf(768, 12, 6), BlockType.PARALLEL,devConf=DEV_CONF)
testMapper.load_state_dict(torch.load("./weights/mapper_state_dict.pth"))

<All keys matched successfully>

In [42]:
from model.CombinationModel import CombinationModel

In [43]:
cModel = CombinationModel(
    distilBert=bertModel,
    decoder=testMapper,
    outputProject=testOutput)
cModel.eval()

CombinationModel(
  (distilBert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1):

In [44]:
# neg
testInput = bertTokenizer("I think I will make a movie next weekend. Oh wait, I'm working..oh I'm sure I can fit it in. It looks like whoever made this film fit it in. I hope the makers of this crap have day jobs because this film sucked!!! It looks like someones home movie and I don't think more than $100 was spent making it!!! Total crap!!! Who let's this stuff be released?!?!?!",
              return_tensors='pt', padding=True, truncation=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [45]:
cModel(**testInput)

tensor([[9.9906e-01, 9.3980e-04]], grad_fn=<SoftmaxBackward0>)